# Top de Juegos en Google Plat Store

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [2]:
df = pd.read_csv('db/android-games.csv')
df = df.rename(columns={'total ratings':'total_ratings'})
df = df.rename(columns={'growth (30 days)':'growth_30_days'})
df = df.rename(columns={'growth (60 days)':'growth_60_days'})

df

,rank,title,total_ratings,installs,average rating,growth_30_days,growth_60_days,price,category,5 star ratings,4 star ratings,3 star ratings,2 star ratings,1 star ratings,paid
0,1,Garena Free Fire- World Series,86273129,500.0 M,4,2.1,6.9,0.0,GAME ACTION,63546766,4949507,3158756,2122183,12495915,False
1,2,PUBG MOBILE - Traverse,37276732,500.0 M,4,1.8,3.6,0.0,GAME ACTION,28339753,2164478,1253185,809821,4709492,False
2,3,Mobile Legends: Bang Bang,26663595,100.0 M,4,1.5,3.2,0.0,GAME ACTION,18777988,1812094,1050600,713912,4308998,False
3,4,Brawl Stars,17971552,100.0 M,4,1.4,4.4,0.0,GAME ACTION,13018610,1552950,774012,406184,2219794,False
4,5,Sniper 3D: Fun Free Online FPS Shooting Game,14464235,500.0 M,4,0.8,1.5,0.0,GAME ACTION,9827328,2124154,1047741,380670,1084340,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1725,96,زوايا - لعبة ستحرك زوايا عقلك,112408,1.0 M,4,0.9,1.8,0.0,GAME WORD,101036,3607,3237,1229,3297,False
1726,97,Bible Word Puzzle - Free Bible Word Games,111595,1.0 M,4,0.9,2.3,0.0,GAME WORD,88950,14856,4297,1385,2103,False
1727,98,Scrabble® GO - New Word Game,110723,10.0 M,4,0.9,1.9,0.0,GAME WORD,64184,18332,9385,6688,12132,False
1728,99,Word Nut: Word Puzzle Games & Crosswords,109530,5.0 M,4,1.9,4.1,0.0,GAME WORD,99987,4766,1469,953,2353,False


## Observando los datos

### Campos nulots

In [34]:
df.isnull().sum()

rank                   0
title                  0
total_ratings          0
installs_in_million    0
average rating         0
growth_30_days         0
growth_60_days         0
category               0
5 star ratings         0
4 star ratings         0
3 star ratings         0
2 star ratings         0
1 star ratings         0
paid                   0
dtype: int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            1730 non-null   int64  
 1   title           1730 non-null   object 
 2   total_ratings   1730 non-null   int64  
 3   installs        1730 non-null   object 
 4   average rating  1730 non-null   int64  
 5   growth_30_days  1730 non-null   float64
 6   growth_60_days  1730 non-null   float64
 7   price           1730 non-null   float64
 8   category        1730 non-null   object 
 9   5 star ratings  1730 non-null   int64  
 10  4 star ratings  1730 non-null   int64  
 11  3 star ratings  1730 non-null   int64  
 12  2 star ratings  1730 non-null   int64  
 13  1 star ratings  1730 non-null   int64  
 14  paid            1730 non-null   bool   
dtypes: bool(1), float64(3), int64(8), object(3)
memory usage: 191.0+ KB


### Transformando los tipos de datos

Transformamos la cantidad de instalacion de `40 M` a `4000000` millones y el **tipo** Objeto a **Float**

In [4]:
df['installs'] = df.installs.apply(lambda x: float(x.split(" ")[0]) * 1000000 if 'M' in x else float(x.split(' ')[0]) * 1000000).astype('float')
df = df.rename(columns={'installs':'installs_in_million'})
df['installs_in_million'].value_counts()


1.000000e+07    805
5.000000e+07    252
5.000000e+06    245
1.000000e+08    207
1.000000e+06    192
5.000000e+08     27
1.000000e+09      2
Name: installs_in_million, dtype: int64

Veamos las columnas de precio y pago. Decidamos si es necesario continuar con ambas o eliminar una de ellas. 

In [5]:
df['price'].value_counts()

0.00    1723
1.99       3
1.49       1
0.99       1
2.99       1
7.49       1
Name: price, dtype: int64

In [6]:
df['paid'].value_counts()

False    1723
True        7
Name: paid, dtype: int64

- De acuerdo, casi el 99% de los juegos son gratuitos y no hay mucho tamaño de muestra para comparar entre los diferentes rangos de precios.
- El tamaño de la muestra menor a 30, la mayoría de las veces, no cumple con los requisitos mínimos para la muestra - representatividad de la población.
- Para este conjunto de datos, la columna 'precio' no tiene mucho que ofrecer para un análisis más detallado.
Así que dejemos caer la columna de 'precio'.
- Dejar caer una columna, eliminar filas son decisiones que deben tomarse con mucha cautela y deben basarse en el análisis y el conocimiento del dominio. 

In [7]:
df.drop('price', axis=1, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rank                 1730 non-null   int64  
 1   title                1730 non-null   object 
 2   total_ratings        1730 non-null   int64  
 3   installs_in_million  1730 non-null   float64
 4   average rating       1730 non-null   int64  
 5   growth_30_days       1730 non-null   float64
 6   growth_60_days       1730 non-null   float64
 7   category             1730 non-null   object 
 8   5 star ratings       1730 non-null   int64  
 9   4 star ratings       1730 non-null   int64  
 10  3 star ratings       1730 non-null   int64  
 11  2 star ratings       1730 non-null   int64  
 12  1 star ratings       1730 non-null   int64  
 13  paid                 1730 non-null   bool   
dtypes: bool(1), float64(3), int64(8), object(2)
memory usage: 177.5+ KB


# Analisis

## Categorias de juegos
Podremos observar que casi tenemos el mismo tamaño de categorias entre todos los juegos.

In [9]:
df['category'].value_counts(normalize=True)

GAME CARD            0.072832
GAME WORD            0.060116
GAME EDUCATIONAL     0.057803
GAME STRATEGY        0.057803
GAME BOARD           0.057803
GAME MUSIC           0.057803
GAME PUZZLE          0.057803
GAME SIMULATION      0.057803
GAME ACTION          0.057803
GAME TRIVIA          0.057803
GAME ROLE PLAYING    0.057803
GAME CASUAL          0.057803
GAME ARCADE          0.057803
GAME SPORTS          0.057803
GAME ADVENTURE       0.057803
GAME RACING          0.057803
GAME CASINO          0.057803
Name: category, dtype: float64

In [10]:
fig = px.histogram(df,
    x='category',
    title='Categorias de Juegos')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

## Top Ratings

In [11]:
df['total_ratings'].describe(include='all')

count    1.730000e+03
mean     1.064332e+06
std      3.429250e+06
min      3.299300e+04
25%      1.759992e+05
50%      4.286065e+05
75%      8.837970e+05
max      8.627313e+07
Name: total_ratings, dtype: float64

In [12]:
fig = px.histogram(df, x= 'total_ratings', title='Total Ratings sobre los Juegos')
fig.show()

In [13]:
fig = px.box(df, x='total_ratings', hover_data=df[['title','category']])
fig.show()

- Como hemos visto en el histograma, muchas de las calificaciones se encuentran en el rango de calificaciones de 0 a 500.000.
- Por otro lado, tenemos bastantes valores atípicos, lo que aumenta la media y la aleja de la mediana.
- Tenemos una distribución muy sesgada, más específicamente una distribución sesgada a la derecha con los posibles valores atípicos en el lado máximo de la distribución. Entonces, para un análisis más detallado, sería bueno recordar eso.
- En este tipo de situaciones, sería una buena idea buscar el enfoque basado en la mediana.
- Se debe usar el valor mediano, en lugar del valor medio, para obtener algunos conocimientos de las distribuciones. 

## Numeros de instalaciones de los juegos

In [14]:
df['installs_in_million'].describe()

count    1.730000e+03
mean     3.368035e+07
std      7.441448e+07
min      1.000000e+06
25%      5.000000e+06
50%      1.000000e+07
75%      5.000000e+07
max      1.000000e+09
Name: installs_in_million, dtype: float64

In [15]:
fig = px.histogram(df, x='installs_in_million', title='Numeros de Instalaciones en Millones de los Juegos')
fig.show()


In [16]:
fig = px.box(df, x='installs_in_million', hover_data=df[['title','category']])
fig.update_traces(quartilemethod='inclusive')
fig.show()

- Hemos sesgado correctamente la distribución con posibles valores atípicos.
Candy Crush Saga **con un billones de instalaciones** y Crazy Quiz **con 500 millones de instalaciones** que se muestran en el diagrama de caja.
- Sería una buena idea verificar siempre con el conjunto de datos, en el conjunto de datos tenemos 2 recuentos de 1 un billones de instalaciones y 12 recuentos de 500 millones de instalaciones. Y boxplot nos muestra un ejemplo de este número de instalaciones.
- El tamaño de los valores atípicos definitivamente afecta el valor **medio** y las **distribuciones**.
- La diferencia entre el valor medio y el valor medio es realmente enorme (media = 29,1 millones, mediana = 10 millones)
- Como se mencionó anteriormente, sería una buena idea utilizar un **enfoque basado en la mediana**. 

## Juegos gratis

In [17]:
print(df['paid'].value_counts(normalize=True))
print('-'*60)
print(df['paid'].value_counts())

False    0.995954
True     0.004046
Name: paid, dtype: float64
------------------------------------------------------------
False    1723
True        7
Name: paid, dtype: int64


In [18]:
paid_free = df['paid'].value_counts()
label = ['Gratis','Paga']
fig = px.pie(df, values=paid_free.values, names=label, title='Juegos Gratis y de Pago')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

- Casi todos los juegos (excepto 7 de 1723) en este conjunto de datos son juegos gratuitos
- Bien, después de este punto, podemos profundizar en el conjunto de datos. 

## Total Rating por Categoria

In [19]:
total_rating_by_category = df.groupby('category')['total_ratings'].mean()
total_rating_by_category

category
GAME ACTION          4.011344e+06
GAME ADVENTURE       8.935617e+05
GAME ARCADE          1.793780e+06
GAME BOARD           4.457431e+05
GAME CARD            3.326041e+05
GAME CASINO          3.619031e+05
GAME CASUAL          2.470866e+06
GAME EDUCATIONAL     1.529804e+05
GAME MUSIC           2.163020e+05
GAME PUZZLE          9.466929e+05
GAME RACING          1.139027e+06
GAME ROLE PLAYING    7.087648e+05
GAME SIMULATION      9.341417e+05
GAME SPORTS          1.353829e+06
GAME STRATEGY        1.856570e+06
GAME TRIVIA          2.982217e+05
GAME WORD            3.943603e+05
Name: total_ratings, dtype: float64

In [20]:
fig = px.bar(total_rating_by_category, x=total_rating_by_category.index, y=total_rating_by_category.values, labels={'y':'Total Ratings'})
fig.update_layout(xaxis = {'categoryorder':'total descending'})
fig.show()

- Los juegos en las categorías de acción, casual, estrategia, arcade y deportes están obteniendo considerablemente más calificaciones, que los juegos en las categorías educativas y musicales. 

## Numeros de Juegos Intalados por Ctegorias

In [21]:
install_by_category = df.groupby('category')['installs_in_million'].mean()
install_by_category

category
GAME ACTION          7.410000e+07
GAME ADVENTURE       1.803000e+07
GAME ARCADE          7.161000e+07
GAME BOARD           2.123000e+07
GAME CARD            2.041270e+07
GAME CASINO          1.271000e+07
GAME CASUAL          6.397000e+07
GAME EDUCATIONAL     2.289000e+07
GAME MUSIC           2.947000e+07
GAME PUZZLE          3.621000e+07
GAME RACING          4.675000e+07
GAME ROLE PLAYING    1.408000e+07
GAME SIMULATION      2.771000e+07
GAME SPORTS          3.361000e+07
GAME STRATEGY        2.391000e+07
GAME TRIVIA          4.786000e+07
GAME WORD            1.231731e+07
Name: installs_in_million, dtype: float64

In [22]:
fig = px.bar(install_by_category, 
    x=install_by_category.index, 
    y=install_by_category.values, 
    title='Numeros de juegos instalados por categoria',
    labels={'y': 'Total en millones'}
)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Los juegos en las categorías de acción, arcade y casual se instalan significativamente más que los juegos en las categorías de mundo, casino y juegos de rol. 

## Crecimiento de las categorias

### En 30 dias

In [23]:
growth_by_category_30 = df.groupby('category')['growth_30_days'].mean()
growth_by_category_30

category
GAME ACTION            18.808000
GAME ADVENTURE        259.101000
GAME ARCADE            58.924000
GAME BOARD             34.445000
GAME CARD             746.598413
GAME CASINO          2335.253000
GAME CASUAL            36.020000
GAME EDUCATIONAL      102.455000
GAME MUSIC             24.626000
GAME PUZZLE            44.362000
GAME RACING           207.103000
GAME ROLE PLAYING     209.979000
GAME SIMULATION        13.406000
GAME SPORTS           159.543000
GAME STRATEGY          18.281000
GAME TRIVIA          1079.680000
GAME WORD              22.433654
Name: growth_30_days, dtype: float64

In [24]:
fig = px.bar(growth_by_category_30,
    x=growth_by_category_30.index,
    y=growth_by_category_30,
    title='Crecimiento de las categorias en 30 dias',
    labels={'y':'Crecimiento en 30 dias'}
)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

- Aunque los juegos en las categorías de acción obtienen más calificaciones y se instalaron más que los juegos en las otras categorías, los juegos en la categoría de **casino** tienen **más crecimiento en 30 días**.
- Veamos si lo mismo también es cierto para el crecimiento de 60 días. 

### En 60 dias

In [25]:
growth_by_category_60 = df.groupby('category')['growth_60_days'].mean()
growth_by_category_60

category
GAME ACTION          118.294000
GAME ADVENTURE         6.084000
GAME ARCADE           21.970000
GAME BOARD           587.891000
GAME CARD            555.337302
GAME CASINO            2.193000
GAME CASUAL           14.812000
GAME EDUCATIONAL      14.748000
GAME MUSIC            22.160000
GAME PUZZLE           12.062000
GAME RACING           88.963000
GAME ROLE PLAYING      3.037000
GAME SIMULATION       20.196000
GAME SPORTS            8.492000
GAME STRATEGY        435.440000
GAME TRIVIA            6.180000
GAME WORD             55.725000
Name: growth_60_days, dtype: float64

In [26]:
fig = px.bar(growth_by_category_60,
    x=growth_by_category_60.index,
    y=growth_by_category_60,
    title='Crecimiento de las categorias en 60 dias',
    labels={'y':'Crecimiento en 60 dias'}
)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

### Conclusion
No, el crecimiento en 60 días para las categorías de juegos en el casino, aventura y juegos de rol es significativamente menor que su crecimiento en 30 días.

Con un conjunto de datos dado, solo podemos especular algo, pero no podemos hacer suposiciones analíticas basadas en los datos dados. Necesitamos más variables para explicar las diferencias significativas para algunas de las categorías en el crecimiento de 30 a 60 días.

Veamos los 3 mejores juegos clasificados en cada categoría en detalle.

## Top 3 de Juegos por Categoria

In [27]:
top_ranked_games = df[df['rank'] < 4][['rank','title','category', 'total_ratings', 'installs_in_million', '5 star ratings']]
top_ranked_games

,rank,title,category,total_ratings,installs_in_million,5 star ratings
0,1,Garena Free Fire- World Series,GAME ACTION,86273129,5.000000e+08,63546766
1,2,PUBG MOBILE - Traverse,GAME ACTION,37276732,5.000000e+08,28339753
2,3,Mobile Legends: Bang Bang,GAME ACTION,26663595,1.000000e+08,18777988
100,1,Roblox,GAME ADVENTURE,21820451,1.000000e+08,16674013
101,2,Pokémon GO,GAME ADVENTURE,14541662,1.000000e+08,9517488
102,3,Criminal Case,GAME ADVENTURE,4273420,1.000000e+08,3264905
200,1,Subway Surfers,GAME ARCADE,35665901,1.000000e+09,27138572
201,2,Hungry Shark Evolution - Offline survival game,GAME ARCADE,7202013,1.000000e+08,5220860
202,3,Geometry Dash Lite,GAME ARCADE,6960814,1.000000e+08,4787054
300,1,Ludo King™,GAME BOARD,7512316,5.000000e+08,5291589


In [28]:
fig = px.scatter(top_ranked_games, 
    y='title', 
    x='total_ratings',
    hover_data=top_ranked_games[['category','rank']],
    color='category',
    title='Top 3 de Juegos por Categoria',
    labels={
        "title": "Juegos",
        "total_ratings": "Total Rating",
    }
)
fig.show()

- Como se mencionó anteriormente, los juegos en las categorías de acción, arcade y casual se instalan significativamente más que los juegos en las categorías de trivia, casino y palabras. 

- Lo mismo es cierto incluso para los juegos mejor clasificados en estas categorías. 

## Los 3 mejores juegos por categoría y sus calificaciones de 5 estrellas 

In [29]:
fig = px.scatter(top_ranked_games, 
    y='title', 
    x='5 star ratings',
    hover_data=top_ranked_games[['category','rank']],
    color='category',
    title='Top 3 de Juegos con 5 Estrellas en Rankings',
    labels={
        "title": "Juegos",
        "5 star ratings": "5 star ratings",
    }
)
fig.show()

Los juegos de las categorías de acción, estrategia, casual y arcade, otras categorias que tienen 5 estrelas son los juegos de simulacion, educativo y entre otros.

## Top 20 juegos

In [30]:
df.columns

Index(['rank', 'title', 'total_ratings', 'installs_in_million',
       'average rating', 'growth_30_days', 'growth_60_days', 'category',
       '5 star ratings', '4 star ratings', '3 star ratings', '2 star ratings',
       '1 star ratings', 'paid'],
      dtype='object')

In [31]:
# rdenamos los valores
top_20 = df.sort_values(by='installs_in_million', ascending=False).head(20)
top_20


,rank,title,total_ratings,installs_in_million,average rating,growth_30_days,growth_60_days,category,5 star ratings,4 star ratings,3 star ratings,2 star ratings,1 star ratings,paid
200,1,Subway Surfers,35665901,1.000000e+09,4,0.5,1.0,GAME ARCADE,27138572,3366600,1622695,814890,2723142,False
626,1,Candy Crush Saga,31367945,1.000000e+09,4,0.9,1.6,GAME CASUAL,23837448,4176798,1534041,486005,1333650,False
0,1,Garena Free Fire- World Series,86273129,5.000000e+08,4,2.1,6.9,GAME ACTION,63546766,4949507,3158756,2122183,12495915,False
1326,1,8 Ball Pool,21632735,5.000000e+08,4,1.2,630.8,GAME SPORTS,16281475,2268294,1017204,425693,1640067,False
1614,89,Cuanto Sabes de la Biblia,45126,5.000000e+08,4,954.5,4.2,GAME TRIVIA,38933,3912,1284,378,617,False
1610,85,90's Quiz Game,47912,5.000000e+08,4,0.0,0.0,GAME TRIVIA,40057,4661,2300,347,545,False
1607,82,לוגוטסט טריוויה: משחק הסמלים והמותגים הגדול בי...,50002,5.000000e+08,4,0.3,0.7,GAME TRIVIA,41823,4249,1669,589,1669,False
1588,63,Trivia Crack (No Ads),75712,5.000000e+08,4,0.7,1.4,GAME TRIVIA,56454,12628,3284,1088,2256,True
1581,56,Pluck It: hairs and emotions,86335,5.000000e+08,4,1.1,2.3,GAME TRIVIA,72958,6777,1971,1139,3487,False
1568,43,Free RBX Master,116248,5.000000e+08,4,2.7,8.1,GAME TRIVIA,84945,7865,4824,2342,16269,False


In [32]:
fig = px.bar(top_20,
    x='title',
    y='installs_in_million',
    hover_data=['rank','5 star ratings'],
    color='category'
)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

- Las categrias y los juegos que tienen mas de 1B de instalaciones son de arcade y casual.
- Algo muy importante existen juegos que no llegan al 1 billon de instalaciones, pero estan dentro de los 500 millones de instalaciones y tienen una puntuacion demasiada alta que puede llegar a tener mas de 60 millones de calificacion total.
- Por ejemplo: Free Fire

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=322df7a5-4b18-4ddb-8f91-c142e3bf7671' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>